In [3]:
import pandas as pd
import contractions
import nltk
from nltk.corpus import stopwords, wordnet

from nltk.stem import WordNetLemmatizer
nltk.download('all')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
import numpy as np
import gensim
import gensim.downloader as api

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\chall\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\chall\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\chall\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\chall\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\chall\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [3]:
import re

In [18]:
data = pd.read_csv('behaviors.tsv', sep = '\t', names=["impressionId","userId","timestamp","click_history","impressions"])

In [19]:
data.head()

,impressionId,userId,timestamp,click_history,impressions
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...


In [20]:
data.shape

(156965, 5)

In [21]:
data['userId'].unique().__len__()

50000

In [22]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = " ".join(sorted([x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']))
    non_click = " ".join(sorted([x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0']))
    return click,non_click

# We can then indexize these two new columns:
data['click'], data['no_clicks'] = zip(*data['impressions'].map(process_impression))

In [23]:
def process_impression2(impression_list):
    list_of_strings = impression_list.split()
    imp_list = sorted([x.split('-')[0] for x in list_of_strings])
    return imp_list

# We can then indexize these two new columns:
data['impList'] = data['impressions'].map(process_impression2)

In [24]:
data.head()

,impressionId,userId,timestamp,click_history,impressions,click,no_clicks,impList
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N55689,N35729,"[N35729, N55689]"
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,N17059,N14592 N20495 N20678 N22407 N33677 N39317 N429...,"[N14592, N17059, N20495, N20678, N22407, N3367..."
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,N23814,N10960 N11817 N12330 N13131 N16844 N23446 N238...,"[N10960, N11817, N12330, N13131, N16844, N2344..."
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,N49685,N27581 N33632 N35729,"[N27581, N33632, N35729, N49685]"
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,N8400,N10413 N11094 N11363 N11967 N12995 N13486 N160...,"[N10413, N11094, N11363, N11967, N12995, N1348..."


In [26]:
def process_impression_list(impression_list):
    list_of_strings = impression_list.split()
    click = sorted([x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1'])
    non_click = sorted([x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0'])
    return click,non_click

In [27]:
data['clickList'], data['noClickList'] = zip(*data['impressions'].map(process_impression_list))

In [28]:
data.head()

,impressionId,userId,timestamp,click_history,impressions,click,no_clicks,impList,clickList,noClickList
0,1,U13740,11/11/2019 9:05:58 AM,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0,N55689,N35729,"[N35729, N55689]",[N55689],[N35729]
1,2,U91836,11/12/2019 6:11:30 PM,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...,N17059,N14592 N20495 N20678 N22407 N33677 N39317 N429...,"[N14592, N17059, N20495, N20678, N22407, N3367...",[N17059],"[N14592, N20495, N20678, N22407, N33677, N3931..."
2,3,U73700,11/14/2019 7:01:48 AM,N10732 N25792 N7563 N21087 N41087 N5445 N60384...,N50014-0 N23877-0 N35389-0 N49712-0 N16844-0 N...,N23814,N10960 N11817 N12330 N13131 N16844 N23446 N238...,"[N10960, N11817, N12330, N13131, N16844, N2344...",[N23814],"[N10960, N11817, N12330, N13131, N16844, N2344..."
3,4,U34670,11/11/2019 5:28:05 AM,N45729 N2203 N871 N53880 N41375 N43142 N33013 ...,N35729-0 N33632-0 N49685-1 N27581-0,N49685,N27581 N33632 N35729,"[N27581, N33632, N35729, N49685]",[N49685],"[N27581, N33632, N35729]"
4,5,U8125,11/12/2019 4:11:21 PM,N10078 N56514 N14904 N33740,N39985-0 N36050-0 N16096-0 N8400-1 N22407-0 N6...,N8400,N10413 N11094 N11363 N11967 N12995 N13486 N160...,"[N10413, N11094, N11363, N11967, N12995, N1348...",[N8400],"[N10413, N11094, N11363, N11967, N12995, N1348..."


In [29]:
data.to_csv("behaviours_list.csv", index = False)

In [12]:
data['no_clicks'].unique().__len__()

123821

In [4]:
news = pd.read_csv('news.tsv',sep = '\t',names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
news["category"].fillna(' ', inplace = True)
news["subcategory"].fillna(' ', inplace = True)
news["title"].fillna(' ', inplace = True)
news["abstract"].fillna(' ', inplace = True)
news["combined"] = news["category"] + " " + news["subcategory"] + " " + news["title"] + " " + news["abstract"]

In [5]:
news.shape

(51282, 9)

In [6]:
news['itemId'].unique().__len__()

51282

In [7]:
news.head()

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities,combined
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],lifestyle lifestyleroyals The Brands Queen Eli...
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",health weightloss 50 Worst Habits For Belly Fa...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",news newsworld The Cost of Trump's Aid Freeze ...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ...",health voices I Was An NBA Wife. Here's How It...
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","health medical How to Get Rid of Skin Tags, Ac..."


In [18]:
def removeURL(column):
    return column.apply(lambda x: re.sub(r'http\S+', '', x))
def toLetter(column):
    return column.apply(lambda x : re.sub('[^a-zA-Z]',' ',x))

def expandContractions(column):
  contractions_expanded = column.apply(lambda x: [contractions.fix(word) for word in x.split()])
  return [' '.join(map(str, x)) for x in contractions_expanded]

def lowerCase(column):
  return column.apply(lambda x: [word.lower() for word in x])

def tokenize(data_column):
  print(np.shape(data_column))
  return data_column.apply(word_tokenize)

def remove_stopwords(data_column):
  stop_words = set.union(set(stopwords.words('english')),set(stopwords.words('arabic')),set(stopwords.words('french')),set(stopwords.words('german')),set(stopwords.words('italian')),set(stopwords.words('russian')),set(stopwords.words('spanish')))
  return data_column.apply(lambda x: [word for word in x if word not in stop_words])

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        
def pos_tags(data_column):
  return data_column.apply(nltk.tag.pos_tag)

def to_wordnet(data_column):
  return data_column.apply(lambda words: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in words])
def word_lemmatizer(data_column):
  wnl = WordNetLemmatizer()
  return data_column.apply(lambda words: [wnl.lemmatize(word, tag) for word, tag in words])

In [19]:
def pre_processing(data, df):
  data=removeURL(data)
  data=toLetter(data)
  df['sentences'] = expandContractions(data)
  data=tokenize(df['sentences'])
  df.drop('sentences', axis='columns', inplace=True)
  data = lowerCase(data)
  data = remove_stopwords(data)
  data = pos_tags(data)
  data = to_wordnet(data)
  return word_lemmatizer(data)

In [20]:
news['processed']=pre_processing(news['combined'],news)

(51282,)


In [21]:
news.head()

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities,combined,processed
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],lifestyle lifestyleroyals The Brands Queen Eli...,"[lifestyle, lifestyleroyals, brand, queen, eli..."
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",health weightloss 50 Worst Habits For Belly Fa...,"[health, weightloss, bad, habit, belly, fat, s..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",news newsworld The Cost of Trump's Aid Freeze ...,"[news, newsworld, cost, trump, aid, freeze, tr..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ...",health voices I Was An NBA Wife. Here's How It...,"[health, voice, nba, wife, affect, mental, hea..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","health medical How to Get Rid of Skin Tags, Ac...","[health, medical, get, rid, skin, tag, accord,..."


In [22]:
news['processed'].isna().sum()

0

In [22]:
# lemmatizer = WordNetLemmatizer()
news['processed2'] = news['processed'].apply(lambda words : ' '.join([w for w in words]))


In [23]:
news.head()

,itemId,category,subcategory,title,abstract,url,title_entities,abstract_entities,combined,processed,processed2
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[],lifestyle lifestyleroyals The Brands Queen Eli...,"[lifestyle, lifestyleroyals, brand, queen, eli...",lifestyle lifestyleroyals brand queen elizabet...
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...",health weightloss 50 Worst Habits For Belly Fa...,"[health, weightloss, bad, habit, belly, fat, s...",health weightloss bad habit belly fat seemingl...
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId...",news newsworld The Cost of Trump's Aid Freeze ...,"[news, newsworld, cost, trump, aid, freeze, tr...",news newsworld cost trump aid freeze trench uk...
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ...",health voices I Was An NBA Wife. Here's How It...,"[health, voice, nba, wife, affect, mental, hea...",health voice nba wife affect mental health fel...
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","health medical How to Get Rid of Skin Tags, Ac...","[health, medical, get, rid, skin, tag, accord,...",health medical get rid skin tag accord dermato...


In [24]:
news.to_csv("processed_behaviour.csv")